In [1]:
import numpy as np
from os import listdir

核函数转换

In [2]:
def kernelTrans(X, A, kTup):
    m, n = np.shape(X)
    K = np.mat(np.zeros((m, 1)))
    if kTup[0] == 'lin':
        K = X * A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j, :] - A
            K[j] = deltaRow * deltaRow.T
        K = np.exp(K / (-1 * kTup[1]**2))
    else:
        raise NameError('Houston We Have a Problem That Kernel is not recongnized')
    return K

Platt SMO支持函数

In [3]:
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m, 2)))    # 误差缓存
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:, i] = kernelTrans(self.X, self.X[i, :], kTup)
            
# 计算E值并返回
def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T * oS.K[:, k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

# 选择内循环的alpha值
def selectJ(i, oS, Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1, Ei]
    validecachelist = np.nonzero(oS.eCache[:, 0].A)[0]    # 返回非0的E值对应的alpha值
    if len(validecachelist) > 1:
        for k in validecachelist:
            if k == i:
                continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if deltaE > maxDeltaE:    # 选择具有最大步长的j
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]

# i是alpha下标，m是alpha数目
def selectJrand(i, m):
    j = i;
    while j == i:
        j = int(np.random.uniform(0, m))
    return j

# 调整大于H或小于L的alpha值
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

Platt SMO优化例程

In [4]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] >0)):
        j ,Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        # 保证alpha在0与C之间
        if oS.labelMat[i] != oS.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] - oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] - oS.alphas[i])
        if L == H:
            print('L == H')
            return 0
        # alpha[j]的最优修改量
        eta = 2.0 * oS.K[i, j] - oS.K[i, i] - oS.K[j, j]
        if eta >= 0:
            print('eta >= 0')
            return 0
        # 计算新的alpha[j]
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, i)
        # 设置常数项b
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, i] - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[i, j]
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, j] - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[j, j]
        if 0 < oS.alphas[i] and oS.C > oS.alphas[i]:
            oS.b = b1
        elif 0 < oS.alphas[j] and oS.C > oS.alphas[j]:
            oS.b = b2
        else:
            oS.b = (b1 + b2)/2.0
        return 1
    else:
        return 0

Platt SMO外循环代码

In [5]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).transpose(), C, toler, kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while iter < maxIter and (alphaPairsChanged > 0 or entireSet):
        alphaPairsChanged = 0
        if entireSet:    # 遍历所有值
            for i in range(oS.m):
                alphaPairsChanged += innerL(i,oS)
                print('fullSet, iter: %d i: %d, pairs changed %d' % (iter, i, alphaPairsChanged))
            iter += 1
        else:    # 遍历非边界值
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print('non-bound, iter: %d i: %d, pairs changed %d' % (iter, i, alphaPairsChanged))
            iter += 1
        if entireSet:
            entireSet = False
        elif alphaPairsChanged == 0:
            entireSet = True
        print('iteration number: %d' % iter)
    return oS.b, oS.alphas

计算w

In [6]:
def calcWs(alphas, dataArr, classLabels):
    X = np.mat(dataArr)
    labelMat = np.mat(classLabels).transpose()
    m, n = np.shape(X)
    w = np.zeros((n, 1))
    for i in range(m):
        w += np.multiply(alphas[i] * labelMat[i], X[i, :].T)
    return w

将文件信息读取为向量 32\*32 ——> 1\*1024

In [7]:
def img2vector(filename):
    # 创建一个 1*1024 的的数组 
    returnVect = np.zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        # 读取文件
        lineStr = fr.readline()
        for j in range(32):
            # 存储每行前32个字符值
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

In [8]:
def loadImages(dirName):
    hwLabels = []
    trainingFileList = listdir(dirName)
    m = len(trainingFileList)
    trainingMat = np.zeros((m, 1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9:
            hwLabels.append(-1)
        else:
            hwLabels.append(1)
        trainingMat[i, :] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels

def testDigits(kTup=('rbf', 10)):
    dataArr, labelArr = loadImages('digits/trainingDigits')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 1000, kTup)
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    svInd = np.nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print('there are %d Support Vectors' % np.shape(sVs)[0])
    m, n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], kTup)
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print('the training error rate is: %f' % (float(errorCount)/m))
    dataArr, labelArr = loadImages('digits/testDigits')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    m, n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], kTup)
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print('the test error rate is: %f' % (float(errorCount)/m))

In [ ]:
testDigits(('rbf', 20))

L == H
fullSet, iter: 0 i: 0, pairs changed 0
fullSet, iter: 0 i: 1, pairs changed 1
L == H
fullSet, iter: 0 i: 2, pairs changed 1
L == H
fullSet, iter: 0 i: 3, pairs changed 1
L == H
fullSet, iter: 0 i: 4, pairs changed 1
L == H
fullSet, iter: 0 i: 5, pairs changed 1
L == H
fullSet, iter: 0 i: 6, pairs changed 1
L == H
fullSet, iter: 0 i: 7, pairs changed 1
L == H
fullSet, iter: 0 i: 8, pairs changed 1
L == H
fullSet, iter: 0 i: 9, pairs changed 1
L == H
fullSet, iter: 0 i: 10, pairs changed 1
L == H
fullSet, iter: 0 i: 11, pairs changed 1
L == H
fullSet, iter: 0 i: 12, pairs changed 1
L == H
fullSet, iter: 0 i: 13, pairs changed 1
L == H
fullSet, iter: 0 i: 14, pairs changed 1
L == H
fullSet, iter: 0 i: 15, pairs changed 1
L == H
fullSet, iter: 0 i: 16, pairs changed 1
L == H
fullSet, iter: 0 i: 17, pairs changed 1
L == H
fullSet, iter: 0 i: 18, pairs changed 1
L == H
fullSet, iter: 0 i: 19, pairs changed 1
L == H
fullSet, iter: 0 i: 20, pairs changed 1
L == H
fullSet, iter: 0 i: 21,

L == H
fullSet, iter: 0 i: 176, pairs changed 1
L == H
fullSet, iter: 0 i: 177, pairs changed 1
L == H
fullSet, iter: 0 i: 178, pairs changed 1
L == H
fullSet, iter: 0 i: 179, pairs changed 1
L == H
fullSet, iter: 0 i: 180, pairs changed 1
L == H
fullSet, iter: 0 i: 181, pairs changed 1
L == H
fullSet, iter: 0 i: 182, pairs changed 1
L == H
fullSet, iter: 0 i: 183, pairs changed 1
L == H
fullSet, iter: 0 i: 184, pairs changed 1
L == H
fullSet, iter: 0 i: 185, pairs changed 1
L == H
fullSet, iter: 0 i: 186, pairs changed 1
L == H
fullSet, iter: 0 i: 187, pairs changed 1
L == H
fullSet, iter: 0 i: 188, pairs changed 1
L == H
fullSet, iter: 0 i: 189, pairs changed 1
L == H
fullSet, iter: 0 i: 190, pairs changed 1
L == H
fullSet, iter: 0 i: 191, pairs changed 1
L == H
fullSet, iter: 0 i: 192, pairs changed 1
L == H
fullSet, iter: 0 i: 193, pairs changed 1
L == H
fullSet, iter: 0 i: 194, pairs changed 1
L == H
fullSet, iter: 0 i: 195, pairs changed 1
L == H
fullSet, iter: 0 i: 196, pairs ch

L == H
fullSet, iter: 0 i: 354, pairs changed 1
L == H
fullSet, iter: 0 i: 355, pairs changed 1
L == H
fullSet, iter: 0 i: 356, pairs changed 1
L == H
fullSet, iter: 0 i: 357, pairs changed 1
L == H
fullSet, iter: 0 i: 358, pairs changed 1
L == H
fullSet, iter: 0 i: 359, pairs changed 1
L == H
fullSet, iter: 0 i: 360, pairs changed 1
L == H
fullSet, iter: 0 i: 361, pairs changed 1
L == H
fullSet, iter: 0 i: 362, pairs changed 1
L == H
fullSet, iter: 0 i: 363, pairs changed 1
L == H
fullSet, iter: 0 i: 364, pairs changed 1
L == H
fullSet, iter: 0 i: 365, pairs changed 1
L == H
fullSet, iter: 0 i: 366, pairs changed 1
L == H
fullSet, iter: 0 i: 367, pairs changed 1
L == H
fullSet, iter: 0 i: 368, pairs changed 1
L == H
fullSet, iter: 0 i: 369, pairs changed 1
L == H
fullSet, iter: 0 i: 370, pairs changed 1
L == H
fullSet, iter: 0 i: 371, pairs changed 1
L == H
fullSet, iter: 0 i: 372, pairs changed 1
L == H
fullSet, iter: 0 i: 373, pairs changed 1
L == H
fullSet, iter: 0 i: 374, pairs ch

L == H
fullSet, iter: 0 i: 528, pairs changed 1
L == H
fullSet, iter: 0 i: 529, pairs changed 1
L == H
fullSet, iter: 0 i: 530, pairs changed 1
L == H
fullSet, iter: 0 i: 531, pairs changed 1
L == H
fullSet, iter: 0 i: 532, pairs changed 1
L == H
fullSet, iter: 0 i: 533, pairs changed 1
L == H
fullSet, iter: 0 i: 534, pairs changed 1
L == H
fullSet, iter: 0 i: 535, pairs changed 1
L == H
fullSet, iter: 0 i: 536, pairs changed 1
L == H
fullSet, iter: 0 i: 537, pairs changed 1
L == H
fullSet, iter: 0 i: 538, pairs changed 1
L == H
fullSet, iter: 0 i: 539, pairs changed 1
L == H
fullSet, iter: 0 i: 540, pairs changed 1
L == H
fullSet, iter: 0 i: 541, pairs changed 1
L == H
fullSet, iter: 0 i: 542, pairs changed 1
L == H
fullSet, iter: 0 i: 543, pairs changed 1
L == H
fullSet, iter: 0 i: 544, pairs changed 1
L == H
fullSet, iter: 0 i: 545, pairs changed 1
L == H
fullSet, iter: 0 i: 546, pairs changed 1
L == H
fullSet, iter: 0 i: 547, pairs changed 1
L == H
fullSet, iter: 0 i: 548, pairs ch

L == H
fullSet, iter: 0 i: 701, pairs changed 1
L == H
fullSet, iter: 0 i: 702, pairs changed 1
L == H
fullSet, iter: 0 i: 703, pairs changed 1
L == H
fullSet, iter: 0 i: 704, pairs changed 1
L == H
fullSet, iter: 0 i: 705, pairs changed 1
L == H
fullSet, iter: 0 i: 706, pairs changed 1
L == H
fullSet, iter: 0 i: 707, pairs changed 1
L == H
fullSet, iter: 0 i: 708, pairs changed 1
L == H
fullSet, iter: 0 i: 709, pairs changed 1
L == H
fullSet, iter: 0 i: 710, pairs changed 1
L == H
fullSet, iter: 0 i: 711, pairs changed 1
L == H
fullSet, iter: 0 i: 712, pairs changed 1
L == H
fullSet, iter: 0 i: 713, pairs changed 1
L == H
fullSet, iter: 0 i: 714, pairs changed 1
L == H
fullSet, iter: 0 i: 715, pairs changed 1
L == H
fullSet, iter: 0 i: 716, pairs changed 1
L == H
fullSet, iter: 0 i: 717, pairs changed 1
L == H
fullSet, iter: 0 i: 718, pairs changed 1
L == H
fullSet, iter: 0 i: 719, pairs changed 1
L == H
fullSet, iter: 0 i: 720, pairs changed 1
L == H
fullSet, iter: 0 i: 721, pairs ch

...

non-bound, iter: 66 i: 196, pairs changed 5
non-bound, iter: 66 i: 266, pairs changed 6
non-bound, iter: 66 i: 294, pairs changed 7
non-bound, iter: 66 i: 306, pairs changed 8
non-bound, iter: 66 i: 316, pairs changed 9
non-bound, iter: 66 i: 348, pairs changed 10
non-bound, iter: 66 i: 349, pairs changed 11
non-bound, iter: 66 i: 350, pairs changed 12
non-bound, iter: 66 i: 386, pairs changed 13
non-bound, iter: 66 i: 459, pairs changed 14
non-bound, iter: 66 i: 482, pairs changed 15
non-bound, iter: 66 i: 484, pairs changed 16
non-bound, iter: 66 i: 540, pairs changed 17
non-bound, iter: 66 i: 575, pairs changed 18
non-bound, iter: 66 i: 616, pairs changed 19
non-bound, iter: 66 i: 632, pairs changed 20
non-bound, iter: 66 i: 690, pairs changed 21
non-bound, iter: 66 i: 760, pairs changed 22
non-bound, iter: 66 i: 824, pairs changed 23
non-bound, iter: 66 i: 826, pairs changed 24
non-bound, iter: 66 i: 834, pairs changed 25
non-bound, iter: 66 i: 888, pairs changed 26
non-bound, iter

non-bound, iter: 69 i: 1235, pairs changed 37
non-bound, iter: 69 i: 1256, pairs changed 38
non-bound, iter: 69 i: 1284, pairs changed 39
non-bound, iter: 69 i: 1324, pairs changed 40
non-bound, iter: 69 i: 1326, pairs changed 41
non-bound, iter: 69 i: 1357, pairs changed 42
non-bound, iter: 69 i: 1369, pairs changed 43
non-bound, iter: 69 i: 1370, pairs changed 44
non-bound, iter: 69 i: 1433, pairs changed 45
non-bound, iter: 69 i: 1497, pairs changed 46
non-bound, iter: 69 i: 1530, pairs changed 47
non-bound, iter: 69 i: 1533, pairs changed 48
non-bound, iter: 69 i: 1702, pairs changed 49
non-bound, iter: 69 i: 1933, pairs changed 50
iteration number: 70
non-bound, iter: 70 i: 13, pairs changed 1
non-bound, iter: 70 i: 125, pairs changed 2
non-bound, iter: 70 i: 126, pairs changed 3
non-bound, iter: 70 i: 128, pairs changed 4
non-bound, iter: 70 i: 196, pairs changed 5
non-bound, iter: 70 i: 266, pairs changed 6
non-bound, iter: 70 i: 294, pairs changed 7
non-bound, iter: 70 i: 306, 

non-bound, iter: 73 i: 540, pairs changed 17
non-bound, iter: 73 i: 575, pairs changed 18
non-bound, iter: 73 i: 616, pairs changed 19
non-bound, iter: 73 i: 632, pairs changed 20
non-bound, iter: 73 i: 690, pairs changed 21
non-bound, iter: 73 i: 760, pairs changed 22
non-bound, iter: 73 i: 824, pairs changed 23
non-bound, iter: 73 i: 826, pairs changed 24
non-bound, iter: 73 i: 834, pairs changed 25
non-bound, iter: 73 i: 888, pairs changed 26
non-bound, iter: 73 i: 890, pairs changed 27
non-bound, iter: 73 i: 894, pairs changed 28
non-bound, iter: 73 i: 921, pairs changed 29
non-bound, iter: 73 i: 922, pairs changed 30
non-bound, iter: 73 i: 943, pairs changed 31
non-bound, iter: 73 i: 948, pairs changed 32
non-bound, iter: 73 i: 950, pairs changed 33
non-bound, iter: 73 i: 1021, pairs changed 34
non-bound, iter: 73 i: 1096, pairs changed 35
non-bound, iter: 73 i: 1112, pairs changed 36
non-bound, iter: 73 i: 1235, pairs changed 37
non-bound, iter: 73 i: 1256, pairs changed 38
non-b

non-bound, iter: 76 i: 1530, pairs changed 47
non-bound, iter: 76 i: 1533, pairs changed 48
non-bound, iter: 76 i: 1702, pairs changed 49
non-bound, iter: 76 i: 1933, pairs changed 50
iteration number: 77
non-bound, iter: 77 i: 13, pairs changed 1
non-bound, iter: 77 i: 125, pairs changed 2
non-bound, iter: 77 i: 126, pairs changed 3
non-bound, iter: 77 i: 128, pairs changed 4
non-bound, iter: 77 i: 196, pairs changed 5
non-bound, iter: 77 i: 266, pairs changed 6
non-bound, iter: 77 i: 294, pairs changed 7
non-bound, iter: 77 i: 306, pairs changed 8
non-bound, iter: 77 i: 316, pairs changed 9
non-bound, iter: 77 i: 348, pairs changed 10
non-bound, iter: 77 i: 349, pairs changed 11
non-bound, iter: 77 i: 350, pairs changed 12
non-bound, iter: 77 i: 386, pairs changed 13
non-bound, iter: 77 i: 459, pairs changed 14
non-bound, iter: 77 i: 482, pairs changed 15
non-bound, iter: 77 i: 484, pairs changed 16
non-bound, iter: 77 i: 540, pairs changed 17
non-bound, iter: 77 i: 575, pairs change

non-bound, iter: 80 i: 890, pairs changed 27
non-bound, iter: 80 i: 894, pairs changed 28
non-bound, iter: 80 i: 921, pairs changed 29
non-bound, iter: 80 i: 922, pairs changed 30
non-bound, iter: 80 i: 943, pairs changed 31
non-bound, iter: 80 i: 948, pairs changed 32
non-bound, iter: 80 i: 950, pairs changed 33
non-bound, iter: 80 i: 1021, pairs changed 34
non-bound, iter: 80 i: 1096, pairs changed 35
non-bound, iter: 80 i: 1112, pairs changed 36
non-bound, iter: 80 i: 1235, pairs changed 37
non-bound, iter: 80 i: 1256, pairs changed 38
non-bound, iter: 80 i: 1284, pairs changed 39
non-bound, iter: 80 i: 1324, pairs changed 40
non-bound, iter: 80 i: 1326, pairs changed 41
non-bound, iter: 80 i: 1357, pairs changed 42
non-bound, iter: 80 i: 1369, pairs changed 43
non-bound, iter: 80 i: 1370, pairs changed 44
non-bound, iter: 80 i: 1433, pairs changed 45
non-bound, iter: 80 i: 1497, pairs changed 46
non-bound, iter: 80 i: 1530, pairs changed 47
non-bound, iter: 80 i: 1533, pairs change